In [ ]:
import pandas as pd
from google.colab import files

class FilmDataProcessor:
    def __init__(self, file_path):
        # Inicializar con la ruta del archivo Excel
        self.file_path = file_path
        self.dataframes = {}

    def load_data(self):
        # Cargar los datos de cada hoja y limpiar los nombres de las columnas
        sheet_names = ['film', 'inventory', 'rental', 'customer', 'store']
        for sheet in sheet_names:
            df = pd.read_excel(self.file_path, sheet_name=sheet)
            df.columns = df.columns.str.strip()  # Eliminar espacios en los nombres de las columnas
            self.dataframes[sheet] = df

    def clean_data(self):
        # Limpieza de cada hoja cargada
        for sheet_name, df in self.dataframes.items():
            if sheet_name == 'film':
                df = self.clean_film(df)
            elif sheet_name == 'inventory':
                df = self.clean_inventory(df)
            elif sheet_name == 'rental':
                df = self.clean_rental(df)
            elif sheet_name == 'customer':
                df = self.clean_customer(df)
            elif sheet_name == 'store':
                df = self.clean_store(df)
            # Actualizar el DataFrame limpio en el diccionario
            self.dataframes[sheet_name] = df

    def clean_film(self, df):
        # Limpieza específica de la hoja 'film'
        df['film_id'] = pd.to_numeric(df['film_id'].astype(str).str.replace(r'[^0-9]', '', regex=True), errors='coerce').astype('Int64')
        df['title'] = df['title'].astype(str).str.strip()
        df['release_year'] = pd.to_numeric(df['release_year'].astype(str).str.replace(r'[^0-9]', '', regex=True), errors='coerce').astype('Int64')
        df['last_update'] = pd.to_datetime(df['last_update'], errors='coerce')
        return df

    def clean_inventory(self, df):
        # Limpieza específica de la hoja 'inventory'
        df['inventory_id'] = pd.to_numeric(df['inventory_id'].astype(str).str.replace(r'[^0-9]', '', regex=True), errors='coerce').astype('Int64')
        df['last_update'] = pd.to_datetime(df['last_update'], errors='coerce')
        return df

    def clean_rental(self, df):
        # Limpieza específica de la hoja 'rental'
        df['rental_id'] = pd.to_numeric(df['rental_id'].astype(str).str.replace(r'[^0-9]', '', regex=True), errors='coerce').astype('Int64')
        df['rental_date'] = pd.to_datetime(df['rental_date'], errors='coerce')
        df['return_date'] = pd.to_datetime(df['return_date'], errors='coerce')
        df['last_update'] = pd.to_datetime(df['last_update'], errors='coerce')
        return df

    def clean_customer(self, df):
        # Limpieza específica de la hoja 'customer'
        df['customer_id'] = pd.to_numeric(df['customer_id'].astype(str).str.replace(r'[^0-9]', '', regex=True), errors='coerce').astype('Int64')
        if 'customer_id_old' in df.columns:
            df = df.drop(columns=['customer_id_old'])
        df['last_update'] = pd.to_datetime(df['last_update'], errors='coerce')
        return df

    def clean_store(self, df):
        # Limpieza específica de la hoja 'store'
        df['store_id'] = pd.to_numeric(df['store_id'].astype(str).str.replace(r'[^0-9]', '', regex=True), errors='coerce').astype('Int64')
        df['last_update'] = pd.to_datetime(df['last_update'], errors='coerce')
        return df

    def combine_data(self):
        # Unir los DataFrames cargados y limpiados
        df_film_inventory = pd.merge(self.dataframes['inventory'], self.dataframes['film'], how='left', on='film_id', suffixes=('_inventory', '_film'))
        df_film_inventory_rental = pd.merge(self.dataframes['rental'], df_film_inventory, how='left', on='inventory_id', suffixes=('_rental', '_film_inventory'))
        df_film_inventory_rental_customer = pd.merge(df_film_inventory_rental, self.dataframes['customer'], how='left', on='customer_id', suffixes=('_rental', '_customer'))

        # Verificar si 'store_id_customer' existe antes de la unión final
        if 'store_id_customer' in df_film_inventory_rental_customer.columns and 'store_id' in self.dataframes['store'].columns:
            df_final = pd.merge(df_film_inventory_rental_customer, self.dataframes['store'], how='left', left_on='store_id_customer', right_on='store_id', suffixes=('_customer', '_store'))
            return df_final
        else:
            raise KeyError("La columna 'store_id_customer' no se encuentra en df_film_inventory_rental_customer o la columna 'store_id' no se encuentra en df_store.")

    def save_and_download(self, df_final, output_file_name):
        # Guardar y descargar el archivo Excel combinado
        output_file_path = f'/content/{output_file_name}.xlsx'
        df_final.to_excel(output_file_path, index=False)
        files.download(output_file_path)

# Uso de la clase FilmDataProcessor
file_path = '/content/Films_2.xlsx'  # Ajustar según la ruta en Google Colab
film_processor = FilmDataProcessor(file_path)

# Cargar, limpiar, combinar, y guardar los datos
film_processor.load_data()
film_processor.clean_data()
df_final = film_processor.combine_data()
film_processor.save_and_download(df_final, 'Final_Combined_DataFrame')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>